In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

from engine_hms_trainer import *
from engine_hms_model import KagglePaths, LocalPaths, ModelConfig

import torch
from torch import nn
import torch.nn.functional as F

import warnings
warnings.filterwarnings('ignore')

/home/shiyi/miniconda3/envs/kaggle/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/shiyi/miniconda3/envs/kaggle/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
paths = KagglePaths if os.path.exists(KagglePaths.OUTPUT_DIR) else LocalPaths
print("Output Dir: ", paths.OUTPUT_DIR)

Output Dir:  ./outputs/


In [3]:
train_easy, train_hard, all_specs, all_eegs = load_kaggle_data(
    paths.TRAIN_CSV, paths.PRE_LOADED_SPECTOGRAMS, paths.PRE_LOADED_EEGS, split_entropy=ModelConfig.SPLIT_ENTROPY)

print(train_easy.shape)
print(train_hard.shape)

# check if contain NaN
print(train_easy.isnull().sum().sum())
print(train_hard.isnull().sum().sum())

display(train_easy.head())
print(" ")
display(train_hard.head())

(13996, 12)
(6187, 12)
0
0


,eeg_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,spectrogram_id,min,max,patient_id,target
0,642382,0.0,0.0,0.0,0.0,0.0,1.0,14960202,1008.0,1032.0,5955,Other
1,751790,0.0,0.0,1.0,0.0,0.0,0.0,618728447,908.0,908.0,38549,GPD
2,778705,0.0,0.0,0.0,0.0,0.0,1.0,52296320,0.0,0.0,40955,Other
3,1629671,1.0,0.0,0.0,0.0,0.0,0.0,2036345030,0.0,160.0,37481,Seizure
4,2061593,0.0,0.0,0.0,0.0,0.0,1.0,320962633,1450.0,1450.0,23828,Other


,eeg_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,spectrogram_id,min,max,patient_id,target
0,568657,0.000000,0.000000,0.250000,0.000000,0.166667,0.583333,789577333,0.0,16.0,20654,Other
1,582999,0.000000,0.857143,0.000000,0.071429,0.000000,0.071429,1552638400,0.0,38.0,20230,LPD
2,1895581,0.076923,0.000000,0.000000,0.000000,0.076923,0.846154,128369999,1138.0,1138.0,47999,Other
3,2482631,0.000000,0.000000,0.133333,0.066667,0.133333,0.666667,978166025,1902.0,1944.0,20606,Other
4,2521897,0.000000,0.000000,0.083333,0.083333,0.333333,0.500000,673742515,0.0,4.0,62117,Other


In [ ]:
# check distribution of targets
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
train_easy["target"].value_counts().plot(kind="bar", ax=axes[0])
train_hard["target"].value_counts().plot(kind="bar", ax=axes[1])
axes[0].set_title("Easy")
axes[1].set_title("Hard")
fig.tight_layout()
plt.show()

In [ ]:
# # calculate class weights for loss function
# class_weights = train_easy["target"].value_counts(normalize=True).sort_index()
# class_weights

In [ ]:
# Rebalance hard samples by Randomly sample from easy and add to hard.
# tgt_to_sample = ['Seizure', 'LRDA', 'GRDA']
# sample_ratio = 0.5 # 50% of easy to add to hard for each target label

# for tgt in tgt_to_sample:
#     easy_sample = train_easy[train_easy["target"] == tgt].sample(frac=sample_ratio)
#     train_hard = pd.concat([train_hard, easy_sample], axis=0)

# print("Hard after adding easy: ", train_hard.shape)

# # check distribution of targets
# fig, axes = plt.subplots(1, 2, figsize=(10, 5))
# train_easy["target"].value_counts().plot(kind="bar", ax=axes[0])
# train_hard["target"].value_counts().plot(kind="bar", ax=axes[1])
# axes[0].set_title("Easy")
# axes[1].set_title("Hard")
# fig.tight_layout()
# plt.show()

In [4]:
# config DualEncoder
ModelConfig.EPOCHS = 1
ModelConfig.BATCH_SIZE = 16
ModelConfig.GRADIENT_ACCUMULATION_STEPS = 2
ModelConfig.MODEL_BACKBONE = 'dual_encoder'
ModelConfig.MODEL_NAME = "DualEncoder_Default_debug"
ModelConfig.USE_KAGGLE_SPECTROGRAMS = True
ModelConfig.USE_EEG_SPECTROGRAMS = True
ModelConfig.REGULARIZATION = None
ModelConfig.AUGMENT = False
ModelConfig.AUGMENTATIONS = []

hms_predictor = HMSPredictor(paths.OUTPUT_DIR, ModelConfig, k_fold=5)

****************************************************************************************************
Script Start: Sun Mar 17 12:48:46 2024
Model Configurations:
SEED: 20
SPLIT_ENTROPY: 5.5
MODEL_NAME: DualEncoder_Default_debug
MODEL_BACKBONE: dual_encoder
BATCH_SIZE: 16
EPOCHS: 1
GRADIENT_ACCUMULATION_STEPS: 2
DROP_RATE: 0.15
DROP_PATH_RATE: 0.25
WEIGHT_DECAY: 0.01
REGULARIZATION: None
USE_KAGGLE_SPECTROGRAMS: True
USE_EEG_SPECTROGRAMS: True
AMP: True
AUGMENT: False
AUGMENTATIONS: []
PRINT_FREQ: 50
FREEZE: False
NUM_FROZEN_LAYERS: 0
NUM_WORKERS: 0
MAX_GRAD_NORM: 10000000.0
MAE_PRETRAINED_WEIGHTS: facebook/vit-mae-base
MAE_HIDDEN_DROPOUT_PROB: 0.05
MAE_ATTENTION_DROPOUT_PROB: 0.05
****************************************************************************************************


In [ ]:
# # Config EfficientNet 
# ModelConfig.EPOCHS = 6
# ModelConfig.BATCH_SIZE = 16
# ModelConfig.GRADIENT_ACCUMULATION_STEPS = 2
# ModelConfig.MODEL_BACKBONE = 'tf_efficientnet_b2'
# ModelConfig.MODEL_NAME = "ENet_b2_xymask_rebalance"
# ModelConfig.USE_KAGGLE_SPECTROGRAMS = True
# ModelConfig.USE_EEG_SPECTROGRAMS = True
# ModelConfig.REGULARIZATION = None
# ModelConfig.AUGMENT = True
# ModelConfig.AUGMENTATIONS = ['xy_masking']

# hms_predictor = HMSPredictor(paths.OUTPUT_DIR, ModelConfig, k_fold=5)

In [ ]:
# # Config ViTMAE
# ModelConfig.EPOCHS = 6
# ModelConfig.MODEL_BACKBONE = 'vit_mae_base'
# ModelConfig.MODEL_NAME = "ViTMAE_base_mlp_dropout_020"
# ModelConfig.AUGMENT = True
# ModelConfig.USE_KAGGLE_SPECTROGRAMS = True
# ModelConfig.USE_EEG_SPECTROGRAMS = True
# ModelConfig.REGULARIZATION = None
# ModelConfig.AUGMENTATIONS = ['xy_masking']
# ModelConfig.MAE_PRETRAINED_WEIGHTS = "./outputs/vit_mae_pretraining/ViTMAE_PreTrained_Best.pth"
# ModelConfig.MAE_HIDDEN_DROPOUT_PROB = 0.1
# ModelConfig.MAE_ATTENTION_DROPOUT_PROB = 0.1
# ModelConfig.DROP_RATE = 0.2

# hms_predictor = HMSPredictor(paths.OUTPUT_DIR, ModelConfig, k_fold=5)

In [5]:
hms_predictor.train_model(train_easy, train_hard, all_specs, all_eegs)

Train Easy: (13996, 12) | Train Hard: (6187, 12)
Fold: 0 || Valid size 6867 
- First Stage 


Train [0]:   0%|          | 0/566 [00:00<?, ?batch/s]

Epoch 1 [0/566] | Train Loss: 0.8966 Grad: 109557.5703 LR: 4.0000e-06 | Elapse: 1.21s
Epoch 1 [50/566] | Train Loss: 0.8335 Grad: 103439.2969 LR: 4.4438e-05 | Elapse: 9.35s
Epoch 1 [100/566] | Train Loss: 0.6111 Grad: 108406.6797 LR: 9.7617e-05 | Elapse: 17.38s
Epoch 1 [150/566] | Train Loss: 0.5655 Grad: 92268.1875 LR: 9.9643e-05 | Elapse: 25.41s
Epoch 1 [200/566] | Train Loss: 0.4285 Grad: 95468.2734 LR: 9.8138e-05 | Elapse: 33.46s
Epoch 1 [250/566] | Train Loss: 0.5289 Grad: 115599.2109 LR: 9.5492e-05 | Elapse: 41.53s
Epoch 1 [300/566] | Train Loss: 0.3450 Grad: 111204.5000 LR: 9.1766e-05 | Elapse: 49.60s
Epoch 1 [350/566] | Train Loss: 0.5016 Grad: 60511.2383 LR: 8.7051e-05 | Elapse: 57.68s
Epoch 1 [400/566] | Train Loss: 0.4430 Grad: 57837.9805 LR: 8.1457e-05 | Elapse: 65.84s
Epoch 1 [450/566] | Train Loss: 0.4506 Grad: 61915.1562 LR: 7.5117e-05 | Elapse: 74.09s
Epoch 1 [500/566] | Train Loss: 0.4052 Grad: 57430.7070 LR: 6.8181e-05 | Elapse: 82.20s
Epoch 1 [550/566] | Train Loss: 

Valid [0]:   0%|          | 0/430 [00:00<?, ?batch/s]

Epoch 1 [0/430] | Valid Loss: 0.3798 | Elapse: 0.07s
Epoch 1 [50/430] | Valid Loss: 0.2669 | Elapse: 3.36s
Epoch 1 [100/430] | Valid Loss: 0.2998 | Elapse: 6.66s
Epoch 1 [150/430] | Valid Loss: 0.6240 | Elapse: 9.96s
Epoch 1 [200/430] | Valid Loss: 0.3500 | Elapse: 13.27s
Epoch 1 [250/430] | Valid Loss: 0.4212 | Elapse: 16.57s
Epoch 1 [300/430] | Valid Loss: 0.4175 | Elapse: 19.89s
Epoch 1 [350/430] | Valid Loss: 0.3526 | Elapse: 23.19s
Epoch 1 [400/430] | Valid Loss: 0.4065 | Elapse: 26.49s


...

Epoch 1 [262/263] | Valid Loss: 0.2378 | Elapse: 17.37s


Fold 1 Valid Loss: (Easy) nan | (Hard) nan
Elapse: 1.15 min 
Fold: 2 || Valid size 3358 
- First Stage 


Train [0]:   0%|          | 0/733 [00:00<?, ?batch/s]

Epoch 1 [0/733] | Train Loss: 0.8153 Grad: 119326.4844 LR: 4.0000e-06 | Elapse: 0.18s
Epoch 1 [50/733] | Train Loss: 0.7748 Grad: 103382.2031 LR: 2.9644e-05 | Elapse: 8.49s


KeyboardInterrupt: 

In [11]:
oof_df = pd.read_csv("./outputs/DualEncoder_Default_debug_oof_2.csv")

oof_df[oof_df.isnull().any(axis=1)]

,eeg_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,spectrogram_id,min,max,patient_id,target,fold,easy_or_hard,seizure_pred,lpd_pred,gpd_pred,lrda_pred,grda_pred,other_pred
6867,645361232,0.000000,0.000000,0.000,0.000000,0.0000,1.000000,48279765,0.0,0.0,28415,Other,1,easy,NaN,NaN,NaN,NaN,NaN,NaN
6868,645492748,0.000000,0.000000,0.000,0.000000,0.0000,1.000000,2089310044,774.0,774.0,61174,Other,1,easy,NaN,NaN,NaN,NaN,NaN,NaN
6869,645544044,1.000000,0.000000,0.000,0.000000,0.0000,0.000000,2041731724,0.0,34.0,14297,Seizure,1,easy,NaN,NaN,NaN,NaN,NaN,NaN
6870,645934941,0.000000,0.000000,0.000,0.000000,0.0000,1.000000,997872679,296.0,346.0,11154,Other,1,easy,NaN,NaN,NaN,NaN,NaN,NaN
6871,646148494,0.000000,1.000000,0.000,0.000000,0.0000,0.000000,1871919869,428.0,428.0,2641,LPD,1,easy,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11058,4244523369,0.000000,0.250000,0.625,0.000000,0.0000,0.125000,1740857415,728.0,768.0,35627,GPD,1,hard,NaN,NaN,NaN,NaN,NaN,NaN
11059,4244523369,0.333333,0.000000,0.000,0.333333,0.0000,0.333333,1740857415,624.0,648.0,35627,Seizure,1,hard,NaN,NaN,NaN,NaN,NaN,NaN
11060,4270940368,0.000000,0.214286,0.000,0.000000,0.0000,0.785714,1305056139,142.0,142.0,54113,Other,1,hard,NaN,NaN,NaN,NaN,NaN,NaN
11061,4271864856,0.000000,0.307692,0.000,0.000000,0.0000,0.692308,1407693286,0.0,0.0,23369,Other,1,hard,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
pd.set_option('display.max_columns', None)
KL_CRITERION = nn.KLDivLoss(reduction='batchmean')
SOFTMAX = nn.Softmax(dim=1)

TARGET2ID = {
    'Seizure': 0,
    'LPD': 1,
    'GPD': 2,
    'LRDA': 3,
    'GRDA': 4,
    'Other': 5
}

from kl_divergence import score as kaggle_score 

def calc_kaggle_score(oof_df):
    submission_df = oof_df[['eeg_id']+TARGETS_PRED].copy()
    submission_df.columns = ['eeg_id'] + TARGETS
    solution_df = oof_df[['eeg_id']+TARGETS].copy()
    return kaggle_score(solution_df, submission_df, 'eeg_id')

def analyze_oof(oof_csv):

    oof_df = pd.read_csv(oof_csv)
    oof_df['target_pred'] = oof_df[TARGETS_PRED].apply(lambda x: np.argmax(x), axis=1)
    oof_df['target_id'] = oof_df[TARGETS].apply(lambda x: np.argmax(x), axis=1)
    
    oof_df["kl_loss"] = oof_df.apply(
    lambda row: 
        KL_CRITERION(
            F.log_softmax(
                    torch.tensor(row[TARGETS_PRED].astype(np.float32)).unsqueeze(0)
                , dim=1
                ), 
            torch.tensor(row[TARGETS].astype(np.float32))
            ).numpy(),
    axis=1)

    oof_df["kl_loss"] = oof_df['kl_loss'].astype(np.float32)

    oof_df[TARGETS_PRED] = SOFTMAX( torch.tensor(oof_df[TARGETS_PRED].values.astype(np.float32)))

    oof_df.head()

    return oof_df

In [ ]:
csv_path = f'./outputs/{ModelConfig.MODEL_NAME}_oof_2.csv'
print("CSV Path: ", csv_path)

oof_df = analyze_oof(csv_path)

print("Kaggle Score: ", calc_kaggle_score(oof_df))
print("Average KL Loss: ", oof_df["kl_loss"].mean())

oof_df.head()

In [ ]:
plot_oof = oof_df.copy()

# plot confusion matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(plot_oof['target_id'], plot_oof['target_pred']) # (y_true, y_pred)
cm = cm / cm.sum(axis=1)[:, np.newaxis]

fig = plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, cmap='Blues', xticklabels=TARGET2ID.keys(), yticklabels=TARGET2ID.keys())
plt.xlabel('Predicted', fontsize=12)
plt.ylabel('True', fontsize=12)
plt.title(csv_path.split('/')[-1].split('.')[0], fontsize=12)
fig.tight_layout()
fig.savefig(f"./outputs/{ModelConfig.MODEL_NAME}/{csv_path.split('/')[-1].split('.')[0]}_CM.png")
plt.show()

In [ ]:
oof_df["kl_loss"].plot(kind='hist', bins=100, title='KL Loss Distribution', figsize=(8, 5))
print(oof_df["kl_loss"].mean())
plt.show()

In [ ]:
# new figure
fig, axes = plt.subplots(6, 5, figsize=(18, 16), sharex=True, sharey=True)

plot_oof = oof_df[oof_df['kl_loss'] > 2]

for row in range(axes.shape[0]):
    row_selects = plot_oof[plot_oof['target_id']==row]
    target_label = BRAIN_ACTIVITY[row]
    for col in range(axes.shape[1]):
        ax = axes[row, col]
        idx = np.random.choice(row_selects.index)
        df_rows = plot_oof.loc[idx]
        ax.plot(df_rows[TARGETS].values , label='True')
        ax.plot(df_rows[TARGETS_PRED].values, label='Pred')
        ax.set_title(f"{idx} | KL: {df_rows['kl_loss']:.4f} ") #
        ax.set_xticks(range(6))
        ax.set_xticklabels(BRAIN_ACTIVITY)
        ax.grid(True)
        ax.legend()
        if col == 0:
            ax.set_ylabel(target_label, fontsize=12)
       
fig.tight_layout()
plt.show()

In [ ]:
csv_path = './outputs/ENet_b2_xymasking_remove_less/ENet_b2_xymasking_remove_less_oof_2.csv'

oof_df = pd.read_csv(csv_path)

oof_df["kl_loss"] = oof_df.apply(
    lambda row: 
        KL_CRITERION(
            F.log_softmax(
                SOFTMAX(
                    torch.tensor(row[TARGETS_PRED].astype(np.float32)).unsqueeze(0)
                    )
                ), 
            torch.tensor(row[TARGETS].astype(np.float32))
            ).numpy(),
    axis=1)

oof_df["kl_loss"] = oof_df['kl_loss'].astype(np.float32)

# y_pred = oof_df[TARGETS_PRED].values.astype(np.float32)
# y_pred_smax = SOFTMAX(torch.tensor(y_pred)).numpy()
# oof_df[TARGETS_PRED] = y_pred_smax

oof_df['target_pred'] = oof_df[TARGETS_PRED].apply(lambda x: np.argmax(x), axis=1)
oof_df['target_id'] = oof_df['target'].map(TARGET2ID)

oof_df.head()

In [ ]:
oof_df["kl_loss"].plot(kind='hist', bins=100, title='KL Loss Distribution', figsize=(8, 5))
print(oof_df["kl_loss"].mean())
plt.show()

In [ ]:
KL_CRITERION(
    torch.log(
        SOFTMAX(
            torch.tensor(oof_df[TARGETS_PRED].values.astype(np.float32))
            )
        ), 
    torch.tensor(oof_df[TARGETS].values.astype(np.float32))
    ).numpy(),

In [ ]:
plot_oof = oof_df.copy()

# plot confusion matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(plot_oof['target_id'], plot_oof['target_pred']) # (y_true, y_pred)
cm = cm / cm.sum(axis=1)[:, np.newaxis]

fig = plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, cmap='Blues', xticklabels=TARGET2ID.keys(), yticklabels=TARGET2ID.keys())
plt.xlabel('Predicted', fontsize=12)
plt.ylabel('True', fontsize=12)
plt.show()

In [ ]:
# new figure
fig, axes = plt.subplots(6, 5, figsize=(18, 16), sharex=True, sharey=True)

plot_oof = oof_df[oof_df['kl_loss'] > 0.2]

for row in range(axes.shape[0]):
    row_selects = plot_oof[plot_oof['target_id']==row]
    target_label = BRAIN_ACTIVITY[row]
    for col in range(axes.shape[1]):
        ax = axes[row, col]
        idx = np.random.choice(row_selects.index)
        df_rows = plot_oof.loc[idx]
        ax.plot(df_rows[TARGETS].values , label='True')
        ax.plot(df_rows[TARGETS_PRED].values, label='Pred')
        ax.set_title(f"{idx} | KL: {df_rows['kl_loss']:.4f} ") #
        ax.set_xticks(range(6))
        ax.set_xticklabels(BRAIN_ACTIVITY)
        ax.grid(True)
        ax.legend()
        if col == 0:
            ax.set_ylabel(target_label, fontsize=12)
       
fig.tight_layout()
plt.show()




In [ ]:
oof_df[oof_df['eeg_id'] == 11127485]

In [ ]:
oof_df[oof_df['eeg_id'].duplicated()] #oof_df.shape #.groupby('eeg_id')['patient_id'].agg(['nunique', 'count']).sort_values(by='count', ascending=False).head(10)

In [ ]:
score_kaggle = oof_df2.loc[:10].apply(lambda row: calc_kaggle_score(row[['eeg_id']+TARGETS], row[['eeg_id']+TARGETS_PRED]), axis=1)
score_kaggle

In [ ]:
submission_df = oof_df2[['eeg_id']+TARGETS_PRED].copy()
submission_df.columns = ['eeg_id'] + TARGETS

solution_df = oof_df2[['eeg_id']+TARGETS].copy()

score_value = kaggle_score(solution_df, submission_df, 'eeg_id')

score_value

In [ ]:
# oof_df1, cv_1 = analyze_oof("./outputs/ENet_b2_xymasking_remove_less/ENet_b2_xymasking_remove_less_oof_1.csv")
# print(cv_1)
# oof_df1.head()